In [1]:
import audformat
import pandas as pd
# segment the data
from inaSpeechSegmenter import Segmenter
from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid
from audformat.utils import to_filewise_index
from audformat import segmented_index


2023-01-25 19:10:54.118070: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 19:10:54.598340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-25 19:10:54.598388: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-25 19:10:56.349407: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
df_sample = pd.read_csv('../age_files/sample_list.csv', index_col='file')

In [3]:
df_sample = df_sample.rename(columns={'sex':'gender'})
df_sample.speaker = df_sample.speaker.astype('str')
df_sample.to_csv('../age_files/ml_age_df.csv')

In [6]:
# Add a column for duration in seconds
def calc_dur(x):
    starts = x[1]
    ends = x[2]
    return (ends - starts).total_seconds()
# add the root folder for audio samples
root = '/home/felix/data/TU/WS2022/ML/age_files/'
df_sample = df_sample.set_index(df_sample.index.to_series().map(lambda x:root+x))
# add start and end times
df_sample = df_sample.set_index(audformat.utils.to_segmented_index(df_sample.index, allow_nat=False))
# compute duration
df_sample['duration'] = df_sample.index.to_series().map(lambda x:calc_dur(x)) 

In [7]:
df_sample.head()

speaker  \
file                                              start  end                                 
/home/felix/data/TU/WS2022/ML/age_files/Natali... 0 days 0 days 00:01:14.492250  NB_1_m_33   
                                                         0 days 00:01:10.339500  NB_2_f_67   
                                                         0 days 00:00:42.519750  NB_3_f_41   
                                                         0 days 00:01:27.354750  NB_4_m_61   
                                                         0 days 00:01:18.718500  NB_5_m_59   

                                                                                gender  \
file                                              start  end                             
/home/felix/data/TU/WS2022/ML/age_files/Natali... 0 days 0 days 00:01:14.492250      m   
                                                         0 days 00:01:10.339500      f   
                                                         0 days 00:00:42.519750      f   
                                                         0 days 00:01:27.354750      m   
                                                         0 days 00:01:18.718500      m   

                                                                                 age  \
file                                              start  end                           
/home/felix/data/TU/WS2022/ML/age_files/Natali... 0 days 0 days 00:01:14.492250   33   
                                                         0 days 00:01:10.339500   67   
                                                         0 days 00:00:42.519750   41   
                                                         0 days 00:01:27.354750   61   
                                                         0 days 00:01:18.718500   59   

                                                                                 duration  
file                                              start  end                               
/home/felix/data/TU/WS2022/ML/age_files/Natali... 0 days 0 days 00:01:14.492250  74.49225  
                                                         0 days 00:01:10.339500  70.33950  
                                                         0 days 00:00:42.519750  42.51975  
                                                         0 days 00:01:27.354750  87.35475  
                                                         0 days 00:01:18.718500  78.71850

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
seg = Segmenter()
def get_segmentation(file):
    print(f'segmenting {file[0]}')
    segmentation = seg(file[0])
    files, starts, ends = [], [], []
    for entry in segmentation:
        kind = entry[0]
        start = entry[1]
        end = entry[2]
        if kind == 'female' or kind == 'male':
            #print (f'{file}, {start}, {end}')
            files.append(file[0])
            starts.append(start)
            ends.append(end)
    seg_index = segmented_index(files, starts, ends)
    return seg_index

def segment_dataframe(df):
    dfs = []
    for file, values in df.iterrows():
        index = get_segmentation(file)
        dfs.append(
            pd.DataFrame(
                values.to_dict(),
                index,
            )
        )
    return audformat.utils.concat(dfs)

df_seg = segment_dataframe(df_sample)


segmenting /home/felix/data/TU/WS2022/ML/age_files/Natalie/NB_1_m_33.wav
14/14 [==============================] - 1s 50ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Natalie/NB_2_w_67.wav
71/71 [==============================] - 4s 52ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Natalie/NB_3_w_41.wav
47/47 [==============================] - 2s 49ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Natalie/NB_4_m_61.wav
129/129 [==============================] - 2s 15ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Natalie/NB_5_m_59.wav
112/112 [==============================] - 2s 14ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Natalie/NB_6_w_60.wav
34/34 [==============================] - 2s 50ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Natalie/NB_7_w_26.wav
59/59 [==============================] - 1s 15ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Natalie/NB_8_w_30.wav
44/44 [===========================

/home/felix/data/TU/WS2022/ML/ina_env/lib/python3.8/site-packages/inaSpeechSegmenter/segmenter.py:61: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
/home/felix/data/TU/WS2022/ML/ina_env/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


46/46 [==============================] - 2s 49ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/magdalena/mm02_f_31.wav
46/46 [==============================] - 2s 51ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/magdalena/mm03_f_82.wav
 4/22 [====>.........................] - ETA: 0s

/home/felix/data/TU/WS2022/ML/ina_env/lib/python3.8/site-packages/inaSpeechSegmenter/segmenter.py:61: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
/home/felix/data/TU/WS2022/ML/ina_env/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


22/22 [==============================] - 1s 52ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/magdalena/mm04_m_60.wav
60/60 [==============================] - 3s 52ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/magdalena/mm05_f_58.wav
48/48 [==============================] - 2s 49ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/magdalena/mm06_m_14.wav
27/27 [==============================] - 1s 48ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/magdalena/mm06_m_34.wav
27/27 [==============================] - 1s 47ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/magdalena/mm06_m_57.wav
61/61 [==============================] - 3s 48ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/magdalena/mm07_m_32.wav
37/37 [==============================] - 2s 49ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/magdalena/mm08_f_31.wav
39/39 [==============================] - 2s 50ms/step
segmenting /home/felix/data/TU/WS2022/ML/ag

/home/felix/data/TU/WS2022/ML/ina_env/lib/python3.8/site-packages/inaSpeechSegmenter/segmenter.py:61: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
/home/felix/data/TU/WS2022/ML/ina_env/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


4/4 [==============================] - 0s 42ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Sarah/sh_sp3_male_47.wav
 5/30 [====>.........................] - ETA: 0s

/home/felix/data/TU/WS2022/ML/ina_env/lib/python3.8/site-packages/inaSpeechSegmenter/segmenter.py:61: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
/home/felix/data/TU/WS2022/ML/ina_env/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


30/30 [==============================] - 0s 15ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Sarah/sh_sp4_female_68.wav
10/10 [==============================] - 0s 45ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Sarah/sh_sp5_female_40.wav
33/33 [==============================] - 1s 15ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Sarah/sh_sp6_female_56.wav
5/5 [==============================] - 0s 43ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Sarah/sh_sp7_female_54.wav
10/10 [==============================] - 0s 45ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Sarah/sh_sp8_male_36.wav
7/7 [==============================] - 0s 46ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Sarah/sh_sp9_female_33.wav
29/29 [==============================] - 1s 48ms/step
segmenting /home/felix/data/TU/WS2022/ML/age_files/Sarah/sh_sp10_male_54.wav
5/5 [==============================] - 0s 39ms/step
segmenting /home/felix/data/TU/

In [9]:
# Add a column for duration in seconds
df_seg['duration'] = df_seg.index.to_series().map(lambda x:calc_dur(x)) 

In [10]:
df_seg.head()

speaker  \
file                                              start                  end                                 
/home/felix/data/TU/WS2022/ML/age_files/Natali... 0 days 00:00:03.440000 0 days 00:00:05.740000  NB_1_m_33   
                                                  0 days 00:00:07.880000 0 days 00:00:13.240000  NB_1_m_33   
                                                  0 days 00:01:10.660000 0 days 00:01:11.640000  NB_1_m_33   
                                                  0 days 00:00:01.940000 0 days 00:00:07.360000  NB_2_f_67   
                                                  0 days 00:00:15.220000 0 days 00:00:23.860000  NB_2_f_67   

                                                                                                gender  \
file                                              start                  end                             
/home/felix/data/TU/WS2022/ML/age_files/Natali... 0 days 00:00:03.440000 0 days 00:00:05.740000      m   
                                                  0 days 00:00:07.880000 0 days 00:00:13.240000      m   
                                                  0 days 00:01:10.660000 0 days 00:01:11.640000      m   
                                                  0 days 00:00:01.940000 0 days 00:00:07.360000      f   
                                                  0 days 00:00:15.220000 0 days 00:00:23.860000      f   

                                                                                                 age  \
file                                              start                  end                           
/home/felix/data/TU/WS2022/ML/age_files/Natali... 0 days 00:00:03.440000 0 days 00:00:05.740000   33   
                                                  0 days 00:00:07.880000 0 days 00:00:13.240000   33   
                                                  0 days 00:01:10.660000 0 days 00:01:11.640000   33   
                                                  0 days 00:00:01.940000 0 days 00:00:07.360000   67   
                                                  0 days 00:00:15.220000 0 days 00:00:23.860000   67   

                                                                                                 duration  
file                                              start                  end                               
/home/felix/data/TU/WS2022/ML/age_files/Natali... 0 days 00:00:03.440000 0 days 00:00:05.740000      2.30  
                                                  0 days 00:00:07.880000 0 days 00:00:13.240000      5.36  
                                                  0 days 00:01:10.660000 0 days 00:01:11.640000      0.98  
                                                  0 days 00:00:01.940000 0 days 00:00:07.360000      5.42  
                                                  0 days 00:00:15.220000 0 days 00:00:23.860000      8.64

In [ ]:
df_seg.duration.plot(kind='hist')

In [11]:
df_seg.to_csv('../age_files/ml_age_segmented.csv')